# Pythonで読み込む

In [20]:
import delta_sharing

In [21]:
profile_file_01 = 'conf/sensor_data_01.share'
profile_file_02 = 'conf/sensor_data_02.share'
profile_file_power = 'conf/power_grid.share'

In [26]:
client01 = delta_sharing.SharingClient(profile_file_01)
client02 = delta_sharing.SharingClient(profile_file_02)
client_power = delta_sharing.SharingClient(profile_file_power)

In [27]:
client01.list_all_tables()

[Table(name='table', share='sensor_data_01', schema='schema')]

In [28]:
client02.list_all_tables()

[Table(name='table', share='sensor_data_02', schema='schema')]

In [29]:
client_power.list_all_tables()

[Table(name='table', share='power_grid_dummydata', schema='schema')]

In [30]:
table01_url = profile_file_01 + '#sensor_data_01.schema.table'
table02_url = profile_file_02 + '#sensor_data_02.schema.table'
table_power_url = profile_file_power + '#power_grid_dummydata.schema.table'

In [14]:
pdf01 = delta_sharing.load_as_pandas(table01_url)
pdf01

,timestamp,id,state
0,2021-01-05 15:00:00,"0011#0x0,0x1","48,29"
1,2021-01-05 15:00:00,"0016#0x0,0x1","49,24"
2,2021-01-05 15:00:00,"0034#0x0,0x1","48,24"
3,2021-01-05 15:00:00,"0040#0x0,0x1","48,28"
4,2021-01-05 15:00:00,"0044#0x0,0x1","48,20"
...,...,...,...
73195,2021-01-06 15:00:00,"0900#0x0,0x1","48,27"
73196,2021-01-06 15:00:00,"0904#0x0,0x1","48,25"
73197,2021-01-06 15:00:00,"0912#0x0,0x1","48,26"
73198,2021-01-06 15:00:00,"0916#0x0,0x1","48,25"


In [15]:
pdf02 = delta_sharing.load_as_pandas(table02_url)
pdf02

,timestamp,id,state
0,2021-01-05 15:00:00,"1000#0x0,0x1","48,25"
1,2021-01-05 15:00:00,"1003#0x0,0x1","49,20"
2,2021-01-05 15:00:00,"1018#0x0,0x1","48,27"
3,2021-01-05 15:00:00,"1039#0x0,0x1","48,23"
4,2021-01-05 15:00:00,"1040#0x0,0x1","48,27"
...,...,...,...
73195,2021-01-06 15:00:00,"1945#0x0,0x1","48,25"
73196,2021-01-06 15:00:00,"1948#0x0,0x1","49,26"
73197,2021-01-06 15:00:00,"1958#0x0,0x1","49,21"
73198,2021-01-06 15:00:00,"1971#0x0,0x1","48,27"


In [31]:
pdf_power = delta_sharing.load_as_pandas(table_power_url)
pdf_power

,timestamp,Measured,Predicted,UseRate,EstimatedSupplyCapacity
0,2021-01-04 15:00:00,1525,1518,0.815072,1871
1,2021-01-04 15:01:00,1530,1518,0.817745,1871
2,2021-01-04 15:02:00,1514,1518,0.809193,1871
3,2021-01-04 15:03:00,1528,1518,0.816676,1871
4,2021-01-04 15:04:00,1524,1518,0.814538,1871
...,...,...,...,...,...
1459,2021-01-05 14:56:00,1656,1694,0.893686,1853
1460,2021-01-05 14:57:00,1656,1694,0.893686,1853
1461,2021-01-05 14:58:00,1642,1694,0.886131,1853
1462,2021-01-05 14:59:00,1660,1694,0.895845,1853


# PySparkで読み込む

In [24]:
sdf01 = delta_sharing.load_as_spark(table01_url)
sdf01.show()

+------------+---------+-------------------+
|     key_str|value_str|          timestamp|
+------------+---------+-------------------+
|0984#0x0,0x1|    49,27|2021-01-06 00:00:00|
|0984#0x0,0x1|    48,26|2021-01-06 01:00:00|
|0984#0x0,0x1|    48,28|2021-01-06 02:00:00|
|0984#0x0,0x1|    48,26|2021-01-06 03:00:00|
|0984#0x0,0x1|    48,26|2021-01-06 04:00:00|
|0984#0x0,0x1|    48,29|2021-01-06 05:00:00|
|0984#0x0,0x1|    49,27|2021-01-06 06:00:00|
|0984#0x0,0x1|    49,29|2021-01-06 07:00:00|
|0984#0x0,0x1|    48,28|2021-01-06 08:00:00|
|0984#0x0,0x1|    49,26|2021-01-06 09:00:00|
|0984#0x0,0x1|    48,28|2021-01-06 10:00:00|
|0984#0x0,0x1|    48,29|2021-01-06 11:00:00|
|0984#0x0,0x1|    48,25|2021-01-06 12:00:00|
|0984#0x0,0x1|    48,23|2021-01-06 13:00:00|
|0984#0x0,0x1|    48,26|2021-01-06 14:00:00|
|0984#0x0,0x1|    49,29|2021-01-06 15:00:00|
|0984#0x0,0x1|    48,27|2021-01-06 16:00:00|
|0984#0x0,0x1|    48,26|2021-01-06 17:00:00|
|0984#0x0,0x1|    49,29|2021-01-06 18:00:00|
|0984#0x0,

In [25]:
sdf02 = delta_sharing.load_as_spark(table02_url)
sdf02.show()

+------------+---------+-------------------+
|     key_str|value_str|          timestamp|
+------------+---------+-------------------+
|1633#0x0,0x1|    48,27|2021-01-06 00:00:00|
|1633#0x0,0x1|    48,26|2021-01-06 01:00:00|
|1633#0x0,0x1|    48,28|2021-01-06 02:00:00|
|1633#0x0,0x1|    48,26|2021-01-06 03:00:00|
|1633#0x0,0x1|    49,26|2021-01-06 04:00:00|
|1633#0x0,0x1|    48,29|2021-01-06 05:00:00|
|1633#0x0,0x1|    48,27|2021-01-06 06:00:00|
|1633#0x0,0x1|    48,29|2021-01-06 07:00:00|
|1633#0x0,0x1|    49,28|2021-01-06 08:00:00|
|1633#0x0,0x1|    48,26|2021-01-06 09:00:00|
|1633#0x0,0x1|    48,28|2021-01-06 10:00:00|
|1633#0x0,0x1|    48,29|2021-01-06 11:00:00|
|1633#0x0,0x1|    48,25|2021-01-06 12:00:00|
|1633#0x0,0x1|    48,23|2021-01-06 13:00:00|
|1633#0x0,0x1|    48,26|2021-01-06 14:00:00|
|1633#0x0,0x1|    48,29|2021-01-06 15:00:00|
|1633#0x0,0x1|    48,27|2021-01-06 16:00:00|
|1633#0x0,0x1|    48,26|2021-01-06 17:00:00|
|1633#0x0,0x1|    48,29|2021-01-06 18:00:00|
|1633#0x0,